In [1]:
#Clinical trial phase wise input data merge
#These tsvs were downloaded phase wise from the clinicaltrials.gov link provided in the case study document

import pandas as pd

early_phase = pd.read_csv("Early_phase.tsv",delimiter='\t',header = 0)
phase1 = pd.read_csv("Phase1.tsv",delimiter='\t',header = 0)
phase2 = pd.read_csv("Phase2.tsv",delimiter='\t',header = 0)
phase3 = pd.read_csv("Phase3.tsv",delimiter='\t',header = 0)
phase4 = pd.read_csv("Phase4.tsv",delimiter='\t',header = 0)
not_applicable = pd.read_csv("Not_Applicable.tsv",delimiter='\t',header = 0)


base = pd.concat([early_phase,phase1,phase2,phase3,phase4,not_applicable])
print (base.shape)
#print(early_phase.head(n=5))

(19533, 27)


In [2]:
for col in base.columns:
    print (col)

Rank
NCT Number
Title
Acronym
Status
Study Results
Conditions
Interventions
Outcome Measures
Sponsor/Collaborators
Gender
Age
Phases
Enrollment
Funded Bys
Study Type
Study Designs
Other IDs
Start Date
Primary Completion Date
Completion Date
First Posted
Results First Posted
Last Update Posted
Locations
Study Documents
URL


In [3]:
base["intervention_list"] = base.apply(lambda x:x['Interventions'].split("|"), axis=1)
#print (base.head(n=5))
interventions = base["intervention_list"].tolist()
#print (interventions[1])
i=0
only_drugs=[]
for intervention in interventions:
    i=i+1
    drug=[]
    for sub in intervention:
        #Excluding non-drugs with the in []
        if (len((sub.split(":")))>1) and (sub.split(":")[0]  in ["Drug", "Genetic" ,"Biological","Combination Product"]) and ("placebo" not in sub.lower()):
            drug.append(sub.split(":")[1])
        else:
            pass
    only_drugs.append(drug)
print (interventions[1])
print (len(interventions))
print (only_drugs[1])
print (len(only_drugs))

['Genetic: SGT-53', 'Radiation: Radiation', 'Drug: Irinotecan', 'Drug: Temozolomide', 'Drug: Bevacizumab']
19533
[' SGT-53', ' Irinotecan', ' Temozolomide', ' Bevacizumab']
19533


In [4]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
import spacy
i=0
search_list=[]
for drug_list in only_drugs:
    i=i+1
    search=[]
    for drug in drug_list:
        tokens=pos_tag(word_tokenize(drug))
        temp=[]
        for tag in tokens:
            if tag[1] in ["NNP","NN","JJ"] and len(tag[0])>3 and len(wordnet.synsets(tag[0].lower()))<=1:
                temp.append(tag[0])
            else:
                pass
        search.append(temp)
    search_list.append(search)
    if i%1000==0:
        print(i)
        #print (search)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [6]:

ids=base["NCT Number"].tolist()

ids_drugs=list(zip(ids,search_list))

#print (ids_drugs[17999])

# The commented out code writes the combination of keywords found for the drug in question, for example metformin hydrochloride
# Currently the code writes two rows for metformin and hydrochloride.
# There are pro and cons of doing it either ways which can be explained over the call

with open ("ct_id_drug.txt","a") as fw:
    i=0
    for combo in ids_drugs:
        try:
            id=combo[0]
            drugs=combo[1]
            for drug in drugs:
                for item in drug:#
                    #fw.write(id+"\t"+" ".join(x for x in drug).strip()+"\n")
                    fw.write(id+"\t"+item.strip()+"\n")
        except:
            i=i+1
        
print ("Exceptions:")
print (i)


Exceptions:
37
